In [1]:
!pip install transformers
!pip install torch

In [2]:
import torch
import os

In [3]:
# Set the API key
os.environ['ACCESS_KEY'] = ''

In [4]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
api_key = os.getenv('ACCESS_KEY')
tokenizer = AutoTokenizer.from_pretrained("mistralai/mathstral-7B-v0.1",use_auth_token=api_key)
model = AutoModelForCausalLM.from_pretrained("mistralai/mathstral-7B-v0.1",use_auth_token=api_key)

/opt/conda/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:778: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/137k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/588k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00004-of-00006.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00005-of-00006.safetensors:   0%|          | 0.00/4.83G [00:00<?, ?B/s]

model-00006-of-00006.safetensors:   0%|          | 0.00/4.26G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [6]:
import pandas as pd

df = pd.read_csv('/kaggle/input/ctibanchdata/cti-mcq.tsv', sep='\t')
df.head()

,URL,Question,Option A,Option B,Option C,Option D,Prompt,GT
0,https://attack.mitre.org/techniques/T1548/,Which of the following mitigations involves pr...,Audit,Execution Prevention,Operating System Configuration,User Account Control,You are given a multiple-choice question (MCQ)...,B
1,https://attack.mitre.org/techniques/T1548/,Which data source is recommended for monitorin...,Command,File,Process,User Account,You are given a multiple-choice question (MCQ)...,D
2,https://attack.mitre.org/techniques/T1548/,What does mitigation ID M1028 suggest to preve...,Limiting privileges of cloud accounts,Preventing unsigned applications from running,Minimizing applications with setuid or setgid ...,Enforcing the highest UAC level,You are given a multiple-choice question (MCQ)...,C
3,https://attack.mitre.org/techniques/T1548/,Which process creation is an indicator of pote...,C:\Windows\System32\services.exe,C:\Windows\System32\cmd.exe,C:\Windows\System32\rundll32.exe,C:\Windows\System32\notepad.exe,You are given a multiple-choice question (MCQ)...,B
4,https://attack.mitre.org/techniques/T1548/,"In a Linux environment, what is recommended to...",Monitor Windows Registry Key Modification,Monitor OS API Execution,Monitor file metadata for setuid or setgid bit...,Audit process metadata changes,You are given a multiple-choice question (MCQ)...,C


In [ ]:
import csv

def generate_response(prompt, model, tokenizer):
    inputs = tokenizer(prompt, return_tensors="pt")
    with torch.no_grad():
        outputs = model.generate(inputs.input_ids, max_length=200)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

def qcm_chat(df, model, tokenizer):
    results = []

    for index, row in df.iterrows():
        question = row['Question']
        options = {
            'A': row['Option A'],
            'B': row['Option B'],
            'C': row['Option C'],
            'D': row['Option D']
        }
        correct_answer = row['GT']
        prompt = f"Question: {question}\nOptions:\nA) {options['A']}\nB) {options['B']}\nC) {options['C']}\nD) {options['D']}\nPlease choose the correct option:"

        response = generate_response(prompt, model, tokenizer)
        model_answer = response.strip().upper()[0]  # Simple extraction logic

        is_correct = model_answer == correct_answer
        results.append({
            'Question': question,
            'Option A': options['A'],
            'Option B': options['B'],
            'Option C': options['C'],
            'Option D': options['D'],
            'Model Answer': model_answer,
            'Correct Answer': correct_answer,
            'Is Correct': is_correct,
            'Model Response': response
        })

        print(f"Processed question {index + 1}/{len(df)}")

    # Save results to CSV file
    keys = results[0].keys()
    with open('model_responses.csv', 'w', newline='') as output_file:
        dict_writer = csv.DictWriter(output_file, fieldnames=keys)
        dict_writer.writeheader()
        dict_writer.writerows(results)

    print("Results saved to model_responses.csv")

if __name__ == "__main__":
    qcm_chat(df, model, tokenizer)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Processed question 1/2500


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Processed question 2/2500


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Processed question 21/2500


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Processed question 22/2500


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Processed question 23/2500
